In [2]:
import pandas as pd

data = pd.read_json('processtimes.json')
data = data.drop(columns=['StationGroup', "UnitIdType"])

data = data.rename(columns={'SerialNumber': 'ItemNumber', 'Value': 'ProcessTime'})
data

,StationName,ItemNumber,ProcessTime,TimeStamp
0,StationName6,10,0.202,2024-10-23 17:19:37.111000+02:00
1,StationName1,1,0.181,2024-10-23 17:19:37.079000+02:00
2,StationName1,14,0.130,2024-10-23 17:19:37.079000+02:00
3,StationName6,20,0.540,2024-10-23 17:19:37.111000+02:00
4,StationName9,27,0.110,2024-10-23 17:19:37.637000+02:00
...,...,...,...,...
114397,StationName144,26410,15.409,2024-10-23 18:22:16.624000+02:00
114398,StationName85,26207,2.188,2024-10-23 18:22:29.926000+02:00
114399,StationName40,26072,0.084,2024-10-23 18:22:32.147000+02:00
114400,StationName42,26122,0.084,2024-10-23 18:22:32.172000+02:00


# Idea: Define a threshold for each machine and compare values

## 1. Median + 1.5 * Inter Quartile Range

In [3]:
# Method 1: Median + IQR
def median_iqr_threshold(group):
    Q1 = group['ProcessTime'].quantile(0.25)
    Q3 = group['ProcessTime'].quantile(0.75)
    IQR = Q3 - Q1
    return group['ProcessTime'].median() + 1.5 * IQR

## 2. Z-Score

In [4]:
# Method 2: Z-score
def zscore_threshold(group):
    mean = group['ProcessTime'].mean()
    std_dev = group['ProcessTime'].std()
    return mean + 2 * std_dev


## 3. 90th percentile

In [5]:
# Method 3: 90th Percentile
def percentile_threshold(group, percentile=0.9):
    return group['ProcessTime'].quantile(percentile)


In [6]:
# Calculate thresholds using functions applied per station
def calculate_thresholds(df):
    thresholds = {}

    # Median + IQR threshold
    df['Q1'] = df.groupby('StationName')['ProcessTime'].transform(lambda x: x.quantile(0.25))
    df['Q3'] = df.groupby('StationName')['ProcessTime'].transform(lambda x: x.quantile(0.75))
    df['Median'] = df.groupby('StationName')['ProcessTime'].transform('median')
    df['IQR'] = df['Q3'] - df['Q1']
    df['Median_IQR_Threshold'] = df['Median'] + 1.5 * df['IQR']
    
    # Z-score threshold
    df['Mean'] = df.groupby('StationName')['ProcessTime'].transform('mean')
    df['Std_Dev'] = df.groupby('StationName')['ProcessTime'].transform('std')
    df['ZScore_Threshold'] = df['Mean'] + 2 * df['Std_Dev']
    
    # 90th Percentile threshold
    df['90Percentile_Threshold'] = df.groupby('StationName')['ProcessTime'].transform(lambda x: x.quantile(0.9))
    
    # 25th Quantile threshold
    df['25Percentile_Threshold'] = df.groupby('StationName')['ProcessTime'].transform(lambda x: x.quantile(0.25))
    
    # Create columns to label "too long" based on each threshold
    df['Too_Long_Median_IQR'] = df['ProcessTime'] > df['Median_IQR_Threshold']
    df['Too_Long_ZScore'] = df['ProcessTime'] > df['ZScore_Threshold']
    df['Too_Long_90Percentile'] = df['ProcessTime'] > df['90Percentile_Threshold']
    df['Too_Long_25Percentile'] = df['ProcessTime'] > df['25Percentile_Threshold']
    
    return df

# Compare results

In the next step, we're comparing the amount of flagged values per method and apply the two metrics "Median Flagged Count" and "Standard Deviation of Flagged Counts" to determine which method is best for the case.

Median Flagged Count: Typical number of flagged events per station. Higher values mean higher sensitivity -> More flagged events

Standard Deviation of Flagged Counts: High SD suggests that a method is more variable, low SD implies more consist behaviour. -> behaviour across stations

In [8]:
# Apply the threshold calculations and flagging function to the dataframe
df = calculate_thresholds(data)

# Summary: Count how often each method flags values as "too long" for each station
summary = df.groupby('StationName').agg(
    Median_IQR_Flagged=('Too_Long_Median_IQR', 'sum'),
    ZScore_Flagged=('Too_Long_ZScore', 'sum'),
    Percentile90_Flagged=('Too_Long_90Percentile', 'sum'),
    Percentile25_Flagged=('Too_Long_25Percentile', 'sum'),
    Median_IQR_Threshold=('Median_IQR_Threshold', 'mean'),
    ZScore_Threshold=('ZScore_Threshold', 'mean'),
    Percentile90_Threshold=('90Percentile_Threshold', 'mean'),
    Percentile25_Threshold=('25Percentile_Threshold', 'mean')
).reset_index()

print("Summary of Flags and Thresholds per Station:")
print(summary)

Summary of Flags and Thresholds per Station:
        StationName  Median_IQR_Flagged  ZScore_Flagged  Percentile90_Flagged  \
0      StationName1                  87              21                    85   
1     StationName10                 254               4                   126   
2    StationName100                  57               9                    52   
3    StationName101                   5               1                    18   
4    StationName102                  34              12                    25   
..              ...                 ...             ...                   ...   
223   StationName95                 330              88                   203   
224   StationName96                  24              14                    21   
225   StationName97                  11               3                     9   
226   StationName98                  20              11                    53   
227   StationName99                  46              20         

In [10]:
# Reusing the summary table created before
# The summary table already has the count of flagged values per station for each method.

# Calculate median and standard deviation of flagged counts across stations
sensitivity_summary = {
    'Method': ['Median + IQR', 'Z-score', '90th Percentile', '25th Percentile'],
    'Median_Flagged_Count': [
        summary['Median_IQR_Flagged'].median(),
        summary['ZScore_Flagged'].median(),
        summary['Percentile90_Flagged'].median(),
        summary['Percentile25_Flagged'].median()
    ],
    'Std_Dev_Flagged_Count': [
        summary['Median_IQR_Flagged'].std(),
        summary['ZScore_Flagged'].std(),
        summary['Percentile90_Flagged'].std(),
        summary['Percentile25_Flagged'].std()
    ]
}

sensitivity_summary_df = pd.DataFrame(sensitivity_summary)
print("Sensitivity Summary Across Stations:")
sensitivity_summary_df.sort_values(by=['Median_Flagged_Count'], ascending=False)


Sensitivity Summary Across Stations:


,Method,Median_Flagged_Count,Std_Dev_Flagged_Count
3,25th Percentile,167.5,454.312871
2,90th Percentile,23.0,60.373382
0,Median + IQR,14.5,70.466902
1,Z-score,5.0,28.849747


### Result interpretation:

The 90th percentile method flags the most events, whereas the Z-score allows more deviation.

# 25th Percentile Comparison

